In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.append("../../")

In [5]:
import pandas as pd
import geopandas as gpd 
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score

from config.config import BASE_PATH, PATH_TO_PATH_CONFIG_FILE
from src.utils import load_paths_from_yaml, replace_base_path
from src.modeling.utils import temporal_train_test_split

In [6]:

paths = load_paths_from_yaml(PATH_TO_PATH_CONFIG_FILE)
paths = replace_base_path(paths, BASE_PATH)

In [13]:
# read in training data
training_data = gpd.read_file(paths["training_data"])
training_data.date = pd.to_datetime(training_data.date)

In [15]:
features = ["date", "ffmc", "farmyard_d", "hiking_ds", "forest_ds", "rail_dens", "elevation", "slope", "aspect", "foresttype", "pop_dens"]
training_data = training_data.loc[:, features + ["fire"]]
training_data.dropna(inplace=True)

In [20]:
X_train, X_test = temporal_train_test_split(training_data, "date", 0.7)
y_train, y_test = X_train["fire"], X_test["fire"]
X_train.drop(columns=["date", "fire"], inplace=True)
X_test.drop(columns=["date", "fire"], inplace=True)

In [21]:
rf = RandomForestClassifier().fit(X_train, y_train)
pred = rf.predict(X_test)

In [22]:
print(accuracy_score(pred, y_test))
print(precision_score(pred, y_test))
print(recall_score(pred, y_test))

0.7621212121212121
0.7459016393442623
0.8100890207715133
